# section 1.2.1

In [1]:
import utils
import torch
import encoder1
import random
import numpy as np
from torch.utils.data import DataLoader, random_split
from utils import plot_tsne
from torchvision import datasets, transforms
import matplotlib.pyplot as plt


In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
autoencoder = encoder1.Autoencoder(mnist=True).to(device)
classifier = encoder1.Classifier(autoencoder.encoder, num_classes=10).to(device)
def freeze_seeds(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

freeze_seeds(0)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
        #one possible convenient normalization. You don't have to use it.
])

train_dataset = datasets.MNIST(root="/datasets/cv_datasets/data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="/datasets/cv_datasets/data", train=False, download=True, transform=transform)

: 

In [ ]:
batch_size=8
validation_split = 0.2
validation_size = int(len(train_dataset) * validation_split)
train_size = len(train_dataset) - validation_size

train_subset, val_subset = random_split(train_dataset, [train_size, validation_size])


# Create data loaders
train_loader = DataLoader(
    train_subset, 
    batch_size=batch_size,
    shuffle=True,
    num_workers=2, 
)

val_loader = DataLoader(
    val_subset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
)

: 

In [ ]:
def test(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f'Accuracy of the network on the test images: {accuracy * 100:.2f}%')
    return accuracy
# Load the model
# load the model
classifier.load_state_dict(torch.load('classifier_MNIST.pth'))

# Set the model to evaluation mode
# Test the model
accuracy = test(classifier, test_loader, device)


# plot the t-SNE visualization
plot_tsne(classifier, test_loader, device)

: 